In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

In [ ]:
import os,sys
sys.path.append('./misc/lib/python3.7/site-packages')

import math
import numpy as np
import requests
import nglview as nv
import ipywidgets as widgets
%matplotlib notebook
import matplotlib.pyplot as plt
from IPython.display import display, display_markdown
from ipywidgets import Layout, HTML
from pathlib import Path

import parmed as pmd
import re

import hublib.use
from hublib.ui import FileUpload, Download
from hublib.cmd import runCommand

from scipy.ndimage import gaussian_filter

import time

%matplotlib notebook
# %use gromacs-2018.4
%use pymol-1.8.4

import hublib.usesh
%usesh gromacs-2018.4

np.set_printoptions(precision=8)
np.set_printoptions(suppress=True)

In [ ]:
topin = 'topol.top'

protfile = 'protein.gro'
pgrofile = 'pigments.gro'

cfile = 'complex.gro'
topout = 'complex.top'

# Build top file
pigstruc = pmd.load_file(pgrofile)
PigList = []
PigTypes = []
for res in pigstruc.residues:
    PigList.append(res.name)
    if PigTypes.count(res.name)==0:
        PigTypes.append(res.name)

with open(topin, 'r') as ifd:
    wrote_include = False
    with open(topout, 'w') as ofd:
        for line in ifd:
            ofd.write(line)
            if line[0:8]=='#include' and wrote_include==False:
                for name in PigTypes:
                    ofd.write('#include \"'+name+'.itp\""\n')
                    wrote_include = True
                    
        for name in PigTypes:
            numstr = str(PigList.count(name))
            while len(numstr)+len(name)<21:
                numstr = " " + numstr
            ofd.write(name + numstr)
        ofd.close()
    ifd.close()
    

# Build gro file
count = -1
NPigAtoms = -1
PigLines = []
with open(pgrofile) as pfd:
    for line in pfd:
        
        # With count<NPigAtoms+1, we include box vector from pigment file.
        # Should be zeros since we just generated the gro file from a PDB. 
        # If NPigAtoms is already set and we aren't done yet.
        if NPigAtoms>0 and count<NPigAtoms+1:
            PigLines.append(line)
            count += 1
        
        if len(line.split())==1 and line.split()[0].isdigit():
            NPigAtoms = int(line.split()[0])
            count = 0
    pfd.close()
    
count = -1
NProAtoms = -1
ProLines = []
HeadLine = ''
with open(protfile) as pfd:
    for line in pfd:
        
        # Record first line as header
        if len(HeadLine)==0:
            HeadLine = line
        
        # With count<NProAtoms, we do NOT include the box vector.
        # If NProAtoms is already set and we aren't done yet.
        if NProAtoms>0 and count<NProAtoms:
            ProLines.append(line)
            count += 1
        
        if len(line.split())==1 and line.split()[0].isdigit():
            NProAtoms = int(line.split()[0])
            count = 0
    pfd.close()
    
        
out = !{"cp " + protfile + " " + cfile}
with open(cfile, 'w') as cfd:
    
    # Add header
    cfd.write(HeadLine)
    
    # Write total number of atoms
    cfd.write(str(NProAtoms+NPigAtoms)+"\n")
    
    # Add protein structure
    cfd.writelines(ProLines)
    
    # Add pigment structure
    cfd.writelines(PigLines)
    cfd.close()
    
